# Model Evaluation Notebook

This notebook provides comprehensive evaluation of trained pricing models including performance metrics, validation, and production readiness assessment.

In [ ]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from data.data_generator import FinancialDataGenerator
from data.feature_engineer import FinancialFeatureEngineer
from models.random_forest import RandomForestPricingModel
from models.neural_network import SwaptionPricingNN
from models.ensemble import WeightedEnsemble
from pricing.analytic import BlackScholesPricer
from utils.visualization import FinancialVisualizer

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Load and Prepare Evaluation Data

In [ ]:
# Generate comprehensive evaluation dataset
print("Generating evaluation dataset...")

generator = FinancialDataGenerator(seed=123)  # Different seed for evaluation
eval_data = generator.generate_option_prices(n_samples=20000)

# Feature engineering
feature_engineer = FinancialFeatureEngineer()
eval_features = feature_engineer.create_option_features(eval_data)

# Select features
feature_cols = [
    'spot_price', 'strike_price', 'time_to_expiry', 'risk_free_rate', 'volatility',
    'moneyness', 'log_moneyness', 'vol_time', 'vol_sqrt_time'
]

X_eval = eval_features[feature_cols]
y_eval = eval_features['call_price']

print(f"Evaluation dataset: {X_eval.shape}")
print(f"Target distribution - Mean: ${y_eval.mean():.4f}, Std: ${y_eval.std():.4f}")
print(f"Price range: ${y_eval.min():.4f} - ${y_eval.max():.4f}")

## 2. Load Trained Models

In [ ]:
# Load or train models for evaluation
print("Loading trained models...")

# Random Forest model
rf_model = RandomForestPricingModel(
    n_estimators=100,
    max_depth=20,
    random_state=42
)

# Neural Network model
scaler = StandardScaler()
X_eval_scaled = pd.DataFrame(
    scaler.fit_transform(X_eval),
    columns=X_eval.columns,
    index=X_eval.index
)

nn_model = SwaptionPricingNN(
    input_dim=X_eval.shape[1],
    hidden_dims=[128, 64, 32],
    dropout_rate=0.2
)

# Ensemble model
base_models = [
    RandomForestPricingModel(n_estimators=50, max_depth=15, random_state=42),
    RandomForestPricingModel(n_estimators=100, max_depth=20, random_state=43),
    RandomForestPricingModel(n_estimators=75, max_depth=18, random_state=44)
]
ensemble_model = WeightedEnsemble(base_models)

# Train models (using a subset for speed)
train_size = 10000
X_train_eval = X_eval[:train_size]
y_train_eval = y_eval[:train_size]
X_train_scaled = X_eval_scaled[:train_size]

print("Training models...")
rf_model.train(X_train_eval, y_train_eval)
nn_model.train(X_train_scaled, y_train_eval, epochs=30, batch_size=64)
ensemble_model.train(X_train_eval, y_train_eval)

print("Models trained successfully")

## 3. Comprehensive Model Evaluation

In [ ]:
def comprehensive_evaluation(model, X, y_true, model_name, is_scaled=False):
    """Perform comprehensive model evaluation"""
    
    # Predictions
    start_time = time.time()
    y_pred = model.predict(X)
    inference_time = time.time() - start_time
    
    # Basic metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # Additional metrics
    residuals = y_pred - y_true
    max_error = np.max(np.abs(residuals))
    median_abs_error = np.median(np.abs(residuals))
    
    # Distribution statistics
    pred_mean = np.mean(y_pred)
    pred_std = np.std(y_pred)
    true_mean = np.mean(y_true)
    true_std = np.std(y_true)
    
    # Bias analysis
    mean_bias = np.mean(residuals)
    median_bias = np.median(residuals)
    
    # Confidence intervals (simple approach)
    residuals_std = np.std(residuals)
    confidence_interval = 1.96 * residuals_std / np.sqrt(len(y_true))
    
    return {
        'model': model_name,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'median_ae': median_abs_error,
        'max_error': max_error,
        'r2': r2,
        'mape': mape,
        'inference_time': inference_time,
        'mean_bias': mean_bias,
        'median_bias': median_bias,
        'confidence_interval': confidence_interval,
        'predictions': y_pred,
        'residuals': residuals,
        'pred_mean': pred_mean,
        'pred_std': pred_std,
        'true_mean': true_mean,
        'true_std': true_std
    }

# Evaluate all models
print("Performing comprehensive evaluation...")

X_test_eval = X_eval[train_size:]
y_test_eval = y_eval[train_size:]
X_test_scaled = X_eval_scaled[train_size:]

models_to_evaluate = [
    (rf_model, X_test_eval, "Random Forest"),
    (nn_model, X_test_scaled, "Neural Network"),
    (ensemble_model, X_test_eval, "Ensemble")
]

evaluation_results = []
for model, X_test, name in models_to_evaluate:
    result = comprehensive_evaluation(model, X_test, y_test_eval, name)
    evaluation_results.append(result)
    
    print(f"\n{name} Evaluation Results:")
    print(f"  RMSE: ${result['rmse']:.4f}")
    print(f"  MAE: ${result['mae']:.4f}")
    print(f"  Median AE: ${result['median_ae']:.4f}")
    print(f"  Max Error: ${result['max_error']:.4f}")
    print(f"  R²: {result['r2']:.4f}")
    print(f"  MAPE: {result['mape']:.2f}%")
    print(f"  Mean Bias: ${result['mean_bias']:.6f}")
    print(f"  Inference Time: {result['inference_time']:.4f}s")

## 4. Benchmark Against Analytical Methods

In [ ]:
# Compare with Black-Scholes analytical pricing
print("Benchmarking against Black-Scholes analytical pricing...")

bs_pricer = BlackScholesPricer()
bs_prices = []

# Calculate BS prices for test set
for _, row in X_test_eval.iterrows():
    bs_price = bs_pricer.call_price(
        spot=row['spot_price'],
        strike=row['strike_price'],
        time=row['time_to_expiry'],
        rate=row['risk_free_rate'],
        vol=row['volatility']
    )
    bs_prices.append(bs_price)

bs_prices = np.array(bs_prices)

# Evaluate Black-Scholes
bs_result = comprehensive_evaluation(
    type('MockModel', (), {'predict': lambda self, X: bs_prices})(),
    X_test_eval, y_test_eval, "Black-Scholes"
)

print(f"\nBlack-Scholes Analytical Results:")
print(f"  RMSE: ${bs_result['rmse']:.4f}")
print(f"  MAE: ${bs_result['mae']:.4f}")
print(f"  R²: {bs_result['r2']:.4f}")
print(f"  MAPE: {bs_result['mape']:.2f}%")

# Add to evaluation results
evaluation_results.append(bs_result)

## 5. Residual Analysis

In [ ]:
# Residual analysis plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Residual Analysis Across Models', fontsize=16)

for i, result in enumerate(evaluation_results):
    if i >= 4:  # Limit to 4 models for display
        break
        
    row = i // 2
    col = i % 2
    
    residuals = result['residuals']
    predictions = result['predictions']
    true_values = y_test_eval
    
    # Residuals vs Fitted
    axes[row, col].scatter(predictions, residuals, alpha=0.6, s=1)
    axes[row, col].axhline(y=0, color='r', linestyle='--', linewidth=2)
    axes[row, col].set_xlabel('Predicted Values')
    axes[row, col].set_ylabel('Residuals')
    axes[row, col].set_title(f'{result["model"]} Residuals')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Q-Q plots for residual normality
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Q-Q Plots for Residual Normality', fontsize=16)

for i, result in enumerate(evaluation_results):
    if i >= 4:
        break
        
    row = i // 2
    col = i % 2
    
    residuals = result['residuals']
    
    # Q-Q plot
    (osm, osr), (slope, intercept, r) = stats.probplot(residuals, dist="norm")
    axes[row, col].plot(osm, osr, 'o', alpha=0.6, markersize=2)
    axes[row, col].plot(osm, slope*osm + intercept, 'r-', linewidth=2)
    axes[row, col].set_xlabel('Theoretical Quantiles')
    axes[row, col].set_ylabel('Sample Quantiles')
    axes[row, col].set_title(f'{result["model"]} Q-Q Plot')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Error Distribution Analysis

In [ ]:
# Error distribution histograms
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Error Distribution Analysis', fontsize=16)

for i, result in enumerate(evaluation_results):
    if i >= 4:
        break
        
    row = i // 2
    col = i % 2
    
    residuals = result['residuals']
    
    # Histogram with KDE
    sns.histplot(residuals, bins=50, alpha=0.7, ax=axes[row, col], stat='density')
    
    # Add normal distribution overlay
    mu, std = np.mean(residuals), np.std(residuals)
    x = np.linspace(residuals.min(), residuals.max(), 100)
    axes[row, col].plot(x, 1/(std * np.sqrt(2 * np.pi)) * np.exp(- (x - mu)**2 / (2 * std**2)), 
                       'r-', linewidth=2, label='Normal fit')
    
    axes[row, col].set_xlabel('Residuals')
    axes[row, col].set_ylabel('Density')
    axes[row, col].set_title(f'{result["model"]} Error Distribution')
    axes[row, col].legend()
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Model Robustness Testing

In [ ]:
# Test models on different market conditions
print("Testing model robustness across market conditions...")

# Create different market scenarios
scenarios = {
    'normal': {'vol_range': (0.1, 0.3), 'time_range': (0.1, 2.0)},
    'high_vol': {'vol_range': (0.3, 0.6), 'time_range': (0.1, 2.0)},
    'long_term': {'vol_range': (0.1, 0.3), 'time_range': (2.0, 5.0)},
    'short_term': {'vol_range': (0.1, 0.3), 'time_range': (0.01, 0.1)}
}

robustness_results = []

for scenario_name, params in scenarios.items():
    print(f"Testing scenario: {scenario_name}")
    
    # Generate scenario-specific data
    scenario_data = generator.generate_option_prices(
        n_samples=2000,
        time_range=params['time_range'],
        vol_range=params['vol_range']
    )
    
    scenario_features = feature_engineer.create_option_features(scenario_data)
    X_scenario = scenario_features[feature_cols]
    y_scenario = scenario_features['call_price']
    
    # Evaluate each model on this scenario
    for model, X_test_format, model_name in models_to_evaluate:
        if 'scaled' in model_name.lower():
            X_test_scenario = pd.DataFrame(
                scaler.transform(X_scenario),
                columns=X_scenario.columns
            )
        else:
            X_test_scenario = X_scenario
            
        result = comprehensive_evaluation(model, X_test_scenario, y_scenario, f"{model_name}_{scenario_name}")
        robustness_results.append(result)

# Summarize robustness results
robustness_df = pd.DataFrame([
    {
        'Model': r['model'].split('_')[0],
        'Scenario': '_'.join(r['model'].split('_')[1:]),
        'RMSE': r['rmse'],
        'MAE': r['mae'],
        'R2': r['r2']
    } for r in robustness_results
])

print("\nRobustness Test Results:")
print(robustness_df.pivot_table(values='RMSE', index='Model', columns='Scenario', aggfunc='mean'))

## 8. Production Readiness Assessment

## 9. Final Evaluation Summary